## **RAG의 핵심, 문서 검색기 Retriever**

In [ ]:
# !pip install -q langchain tiktoken langchain-text-splitters langchain-openai langchain-chroma pypdf

### **Retriever의 기본형, 벡터DB 기반 Retriever**

**Chroma 벡터 DB 기반 기본 유사 문서 검색**

In [1]:
from dotenv import load_dotenv
import os

# .env 파일 로드
load_dotenv('../.env')

# 2. 환경 변수 읽기
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [2]:
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
import os
Chroma().delete_collection()
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

#헌법 PDF 파일 로드
loader = PyPDFLoader(r"../../../data/Langchain-RAG/대한민국헌법(헌법)(제00010호)(19880225).pdf")
pages = loader.load_and_split()

#PDF 파일을 500자 청크로 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(pages)

#ChromaDB에 청크들을 벡터 임베딩으로 저장(OpenAI 임베딩 모델 활용)
db = Chroma.from_documents(docs, OpenAIEmbeddings(model = 'text-embedding-3-small'))

#Chroma를 Retriever로 활용
retriever = db.as_retriever()
retriever.invoke("국회의원의 의무")

[Document(id='46c8c67e-dfd1-442e-86e0-71a8641c14b7', metadata={'creator': 'PyPDF', 'creationdate': '2024-04-01T21:26:24+09:00', 'moddate': '2024-04-01T21:26:24+09:00', 'page': 4, 'producer': 'iText 2.1.7 by 1T3XT', 'total_pages': 14, 'page_label': '5', 'source': '../../../data/Langchain-RAG/대한민국헌법(헌법)(제00010호)(19880225).pdf'}, page_content='③국회의원은 그 지위를 남용하여 국가ㆍ공공단체 또는 기업체와의 계약이나 그 처분에 의하여 재산상의 권리ㆍ이\n익 또는 직위를 취득하거나 타인을 위하여 그 취득을 알선할 수 없다.\n \n제47조 ①국회의 정기회는 법률이 정하는 바에 의하여 매년 1회 집회되며, 국회의 임시회는 대통령 또는 국회재적의원\n4분의 1 이상의 요구에 의하여 집회된다.\n②정기회의 회기는 100일을, 임시회의 회기는 30일을 초과할 수 없다.\n③대통령이 임시회의 집회를 요구할 때에는 기간과 집회요구의 이유를 명시하여야 한다.\n \n제48조 국회는 의장 1인과 부의장 2인을 선출한다.\n \n제49조 국회는 헌법 또는 법률에 특별한 규정이 없는 한 재적의원 과반수의 출석과 출석의원 과반수의 찬성으로 의결\n한다. 가부동수인 때에는 부결된 것으로 본다.\n \n제50조 ①국회의 회의는 공개한다. 다만, 출석의원 과반수의 찬성이 있거나 의장이 국가의 안전보장을 위하여 필요하다\n고 인정할 때에는 공개하지 아니할 수 있다.'),
 Document(id='833d9e45-0f9a-4546-af51-d5ee9071328e', metadata={'producer': 'iText 2.1.7 by 1T3XT', 'page_label': '5', 'moddate': '2024-04-01T21:26

**(1) 유사도 점수도 함께 출력하기**

In [3]:
result_score = db.similarity_search_with_score("국회의원의 의무")
result_r_score = db.similarity_search_with_relevance_scores("국회의원의 의무")

print("[유사 청크 1순위]")
print(result_score[0][0].page_content)

print("\n\n[점수]")
print(result_score[0][1]) # 낮을수록 좋다
print(result_score[3][1])
print(result_r_score[0][1]) # 높을수록 좋다
print(result_r_score[3][1])

[유사 청크 1순위]
③국회의원은 그 지위를 남용하여 국가ㆍ공공단체 또는 기업체와의 계약이나 그 처분에 의하여 재산상의 권리ㆍ이
익 또는 직위를 취득하거나 타인을 위하여 그 취득을 알선할 수 없다.
 
제47조 ①국회의 정기회는 법률이 정하는 바에 의하여 매년 1회 집회되며, 국회의 임시회는 대통령 또는 국회재적의원
4분의 1 이상의 요구에 의하여 집회된다.
②정기회의 회기는 100일을, 임시회의 회기는 30일을 초과할 수 없다.
③대통령이 임시회의 집회를 요구할 때에는 기간과 집회요구의 이유를 명시하여야 한다.
 
제48조 국회는 의장 1인과 부의장 2인을 선출한다.
 
제49조 국회는 헌법 또는 법률에 특별한 규정이 없는 한 재적의원 과반수의 출석과 출석의원 과반수의 찬성으로 의결
한다. 가부동수인 때에는 부결된 것으로 본다.
 
제50조 ①국회의 회의는 공개한다. 다만, 출석의원 과반수의 찬성이 있거나 의장이 국가의 안전보장을 위하여 필요하다
고 인정할 때에는 공개하지 아니할 수 있다.


[점수]
0.8001311421394348
0.8947548270225525
0.4339805528469435
0.36716540678371423


**(2) 검색 결과 수 및 조정**

In [4]:
#유사 청크 1개만 반환
retriever = db.as_retriever(search_kwargs={"k": 5})
retriever.invoke("국회의원의 의무")

[Document(id='46c8c67e-dfd1-442e-86e0-71a8641c14b7', metadata={'creator': 'PyPDF', 'page_label': '5', 'creationdate': '2024-04-01T21:26:24+09:00', 'total_pages': 14, 'moddate': '2024-04-01T21:26:24+09:00', 'source': '../../../data/Langchain-RAG/대한민국헌법(헌법)(제00010호)(19880225).pdf', 'producer': 'iText 2.1.7 by 1T3XT', 'page': 4}, page_content='③국회의원은 그 지위를 남용하여 국가ㆍ공공단체 또는 기업체와의 계약이나 그 처분에 의하여 재산상의 권리ㆍ이\n익 또는 직위를 취득하거나 타인을 위하여 그 취득을 알선할 수 없다.\n \n제47조 ①국회의 정기회는 법률이 정하는 바에 의하여 매년 1회 집회되며, 국회의 임시회는 대통령 또는 국회재적의원\n4분의 1 이상의 요구에 의하여 집회된다.\n②정기회의 회기는 100일을, 임시회의 회기는 30일을 초과할 수 없다.\n③대통령이 임시회의 집회를 요구할 때에는 기간과 집회요구의 이유를 명시하여야 한다.\n \n제48조 국회는 의장 1인과 부의장 2인을 선출한다.\n \n제49조 국회는 헌법 또는 법률에 특별한 규정이 없는 한 재적의원 과반수의 출석과 출석의원 과반수의 찬성으로 의결\n한다. 가부동수인 때에는 부결된 것으로 본다.\n \n제50조 ①국회의 회의는 공개한다. 다만, 출석의원 과반수의 찬성이 있거나 의장이 국가의 안전보장을 위하여 필요하다\n고 인정할 때에는 공개하지 아니할 수 있다.'),
 Document(id='833d9e45-0f9a-4546-af51-d5ee9071328e', metadata={'producer': 'iText 2.1.7 by 1T3XT', 'source': '../../../data/Langchain-RAG/대한민국헌법(헌

**(3) 검색 방식 변경 - MMR**

In [5]:
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

#헌법 PDF 파일 로드
loader = PyPDFLoader(r"../../../data/Langchain-RAG/대한민국헌법(헌법)(제00010호)(19880225).pdf")
pages = loader.load_and_split()

#PDF 파일을 500자 청크로 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(pages)

#ChromaDB에 청크들을 벡터 임베딩으로 저장(OpenAI 임베딩 모델 활용)
db = Chroma.from_documents(docs, OpenAIEmbeddings(model = 'text-embedding-3-small'))

In [6]:
#Chroma를 Retriever로 활용
retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs = {"lambda_mult": 0, "fetch_k":20, "k":3}
)
retriever.invoke("국회의원의 의무")

[Document(id='46c8c67e-dfd1-442e-86e0-71a8641c14b7', metadata={'creator': 'PyPDF', 'page_label': '5', 'source': '../../../data/Langchain-RAG/대한민국헌법(헌법)(제00010호)(19880225).pdf', 'total_pages': 14, 'producer': 'iText 2.1.7 by 1T3XT', 'creationdate': '2024-04-01T21:26:24+09:00', 'moddate': '2024-04-01T21:26:24+09:00', 'page': 4}, page_content='③국회의원은 그 지위를 남용하여 국가ㆍ공공단체 또는 기업체와의 계약이나 그 처분에 의하여 재산상의 권리ㆍ이\n익 또는 직위를 취득하거나 타인을 위하여 그 취득을 알선할 수 없다.\n \n제47조 ①국회의 정기회는 법률이 정하는 바에 의하여 매년 1회 집회되며, 국회의 임시회는 대통령 또는 국회재적의원\n4분의 1 이상의 요구에 의하여 집회된다.\n②정기회의 회기는 100일을, 임시회의 회기는 30일을 초과할 수 없다.\n③대통령이 임시회의 집회를 요구할 때에는 기간과 집회요구의 이유를 명시하여야 한다.\n \n제48조 국회는 의장 1인과 부의장 2인을 선출한다.\n \n제49조 국회는 헌법 또는 법률에 특별한 규정이 없는 한 재적의원 과반수의 출석과 출석의원 과반수의 찬성으로 의결\n한다. 가부동수인 때에는 부결된 것으로 본다.\n \n제50조 ①국회의 회의는 공개한다. 다만, 출석의원 과반수의 찬성이 있거나 의장이 국가의 안전보장을 위하여 필요하다\n고 인정할 때에는 공개하지 아니할 수 있다.'),
 Document(id='08fdd114-9a6b-4141-95f9-8a680d26484e', metadata={'page_label': '4', 'page': 3, 'producer': 'iText 2.1.7 by 1T3XT', 'source': '../../

In [7]:
#Chroma를 Retriever로 활용
retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs = {"lambda_mult": 0, "fetch_k":10, "k":3}
)
retriever.invoke("국회의원의 의무")

[Document(id='46c8c67e-dfd1-442e-86e0-71a8641c14b7', metadata={'creationdate': '2024-04-01T21:26:24+09:00', 'page_label': '5', 'total_pages': 14, 'moddate': '2024-04-01T21:26:24+09:00', 'page': 4, 'source': '../../../data/Langchain-RAG/대한민국헌법(헌법)(제00010호)(19880225).pdf', 'producer': 'iText 2.1.7 by 1T3XT', 'creator': 'PyPDF'}, page_content='③국회의원은 그 지위를 남용하여 국가ㆍ공공단체 또는 기업체와의 계약이나 그 처분에 의하여 재산상의 권리ㆍ이\n익 또는 직위를 취득하거나 타인을 위하여 그 취득을 알선할 수 없다.\n \n제47조 ①국회의 정기회는 법률이 정하는 바에 의하여 매년 1회 집회되며, 국회의 임시회는 대통령 또는 국회재적의원\n4분의 1 이상의 요구에 의하여 집회된다.\n②정기회의 회기는 100일을, 임시회의 회기는 30일을 초과할 수 없다.\n③대통령이 임시회의 집회를 요구할 때에는 기간과 집회요구의 이유를 명시하여야 한다.\n \n제48조 국회는 의장 1인과 부의장 2인을 선출한다.\n \n제49조 국회는 헌법 또는 법률에 특별한 규정이 없는 한 재적의원 과반수의 출석과 출석의원 과반수의 찬성으로 의결\n한다. 가부동수인 때에는 부결된 것으로 본다.\n \n제50조 ①국회의 회의는 공개한다. 다만, 출석의원 과반수의 찬성이 있거나 의장이 국가의 안전보장을 위하여 필요하다\n고 인정할 때에는 공개하지 아니할 수 있다.'),
 Document(id='c81ca02d-8542-444b-8c6b-4d61b7aa28fc', metadata={'page': 5, 'moddate': '2024-04-01T21:26:24+09:00', 'creator': 'PyPDF', 'creationdat

In [8]:
#Chroma를 Retriever로 활용
retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs = {"lambda_mult": 0.4, "fetch_k":20, "k":3}
)
retriever.invoke("국회의원의 의무")

[Document(id='46c8c67e-dfd1-442e-86e0-71a8641c14b7', metadata={'producer': 'iText 2.1.7 by 1T3XT', 'total_pages': 14, 'moddate': '2024-04-01T21:26:24+09:00', 'page_label': '5', 'creationdate': '2024-04-01T21:26:24+09:00', 'page': 4, 'creator': 'PyPDF', 'source': '../../../data/Langchain-RAG/대한민국헌법(헌법)(제00010호)(19880225).pdf'}, page_content='③국회의원은 그 지위를 남용하여 국가ㆍ공공단체 또는 기업체와의 계약이나 그 처분에 의하여 재산상의 권리ㆍ이\n익 또는 직위를 취득하거나 타인을 위하여 그 취득을 알선할 수 없다.\n \n제47조 ①국회의 정기회는 법률이 정하는 바에 의하여 매년 1회 집회되며, 국회의 임시회는 대통령 또는 국회재적의원\n4분의 1 이상의 요구에 의하여 집회된다.\n②정기회의 회기는 100일을, 임시회의 회기는 30일을 초과할 수 없다.\n③대통령이 임시회의 집회를 요구할 때에는 기간과 집회요구의 이유를 명시하여야 한다.\n \n제48조 국회는 의장 1인과 부의장 2인을 선출한다.\n \n제49조 국회는 헌법 또는 법률에 특별한 규정이 없는 한 재적의원 과반수의 출석과 출석의원 과반수의 찬성으로 의결\n한다. 가부동수인 때에는 부결된 것으로 본다.\n \n제50조 ①국회의 회의는 공개한다. 다만, 출석의원 과반수의 찬성이 있거나 의장이 국가의 안전보장을 위하여 필요하다\n고 인정할 때에는 공개하지 아니할 수 있다.'),
 Document(id='f0fab0bc-30dd-4dbd-83a1-4716fa6ffc80', metadata={'page_label': '4', 'moddate': '2024-04-01T21:26:24+09:00', 'source': '../../../data

**일반 유사도 검색 방식**

In [9]:
#Chroma를 Retriever로 활용
retriever = db.as_retriever(search_kwargs = {"k":3})
retriever.get_relevant_documents("국회의원의 의무")


C:\Users\user\AppData\Local\Temp\ipykernel_7296\2706477368.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents("국회의원의 의무")


[Document(id='46c8c67e-dfd1-442e-86e0-71a8641c14b7', metadata={'moddate': '2024-04-01T21:26:24+09:00', 'producer': 'iText 2.1.7 by 1T3XT', 'creator': 'PyPDF', 'page_label': '5', 'total_pages': 14, 'source': '../../../data/Langchain-RAG/대한민국헌법(헌법)(제00010호)(19880225).pdf', 'page': 4, 'creationdate': '2024-04-01T21:26:24+09:00'}, page_content='③국회의원은 그 지위를 남용하여 국가ㆍ공공단체 또는 기업체와의 계약이나 그 처분에 의하여 재산상의 권리ㆍ이\n익 또는 직위를 취득하거나 타인을 위하여 그 취득을 알선할 수 없다.\n \n제47조 ①국회의 정기회는 법률이 정하는 바에 의하여 매년 1회 집회되며, 국회의 임시회는 대통령 또는 국회재적의원\n4분의 1 이상의 요구에 의하여 집회된다.\n②정기회의 회기는 100일을, 임시회의 회기는 30일을 초과할 수 없다.\n③대통령이 임시회의 집회를 요구할 때에는 기간과 집회요구의 이유를 명시하여야 한다.\n \n제48조 국회는 의장 1인과 부의장 2인을 선출한다.\n \n제49조 국회는 헌법 또는 법률에 특별한 규정이 없는 한 재적의원 과반수의 출석과 출석의원 과반수의 찬성으로 의결\n한다. 가부동수인 때에는 부결된 것으로 본다.\n \n제50조 ①국회의 회의는 공개한다. 다만, 출석의원 과반수의 찬성이 있거나 의장이 국가의 안전보장을 위하여 필요하다\n고 인정할 때에는 공개하지 아니할 수 있다.'),
 Document(id='d2e600fb-f258-4279-92c6-ce217bff76aa', metadata={'creationdate': '2024-04-01T21:26:24+09:00', 'producer': 'iText 2.1.7 by 1T3XT', 'p

### **사용자의 쿼리를 재해석하여 검색하다, MultiQueryRetriever**

**Chroma DB에 문서 벡터 저장**

In [10]:
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

#헌법 PDF 파일 로드
loader = PyPDFLoader(r"../../../data/Langchain-RAG/대한민국헌법(헌법)(제00010호)(19880225).pdf")
pages = loader.load_and_split()

#PDF 파일을 500자 청크로 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(pages)

#ChromaDB에 청크들을 벡터 임베딩으로 저장(OpenAI 임베딩 모델 활용)
db = Chroma.from_documents(docs, OpenAIEmbeddings(model = 'text-embedding-3-small'))

**질문을 여러 버전으로 재해석하여 Retriever에 활용**

In [11]:
#```Chroma DB에 대한민국 헌법 PDF 임베딩 변환 및 저장하는 과정은 위 셀에 있습니다```
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI

#질문 문장 question으로 저장
question = "국회의원의 의무는 무엇이 있나요?"
#여러 버전의 질문으로 변환하는 역할을 맡을 LLM 선언
llm = ChatOpenAI(model_name="gpt-4o-mini",
                 temperature = 0)
#MultiQueryRetriever에 벡터DB 기반 Retriever와 LLM 선언
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=db.as_retriever(), llm=llm
)

# 여러 버전의 문장 생성 결과를 확인하기 위한 로깅 과정
import logging
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

#여러 버전 질문 생성 결과와 유사 청크 검색 개수 출력
unique_docs = retriever_from_llm.invoke(input=question)
len(unique_docs)

INFO:langchain.retrievers.multi_query:Generated queries: ['국회의원이 수행해야 하는 의무는 어떤 것들이 있나요?  ', '국회의원의 책임과 의무에 대해 설명해 주세요.  ', '국회의원으로서 지켜야 할 의무는 무엇인가요?']


8

### **문서를 여러 벡터로 재해석하다, MultiVectorRetriever**

**Chroma DB에 문서 벡터 저장**

In [16]:
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryByteStore
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader

loaders = PyPDFLoader(r"../../../data/Langchain-RAG/대한민국헌법(헌법)(제00010호)(19880225).pdf"),
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
docs = text_splitter.split_documents(docs)

**Multi Vector를 만들기 위한 작업**

In [17]:
from langchain.embeddings import HuggingFaceEmbeddings

model_name = "jhgan/ko-sbert-nli"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
embedding = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)                                             # 허깅페이스의 "jhgan/ko-sbert-nli" 모델 로드

vectorstore = Chroma(
    collection_name="full_documents", embedding_function=embedding
)
# 상위 문서 저장 위한 레이어 선언
store = InMemoryByteStore()                   # InMemoryByteStore는 상위청크를 저장하고 이를 하위청크와 연결할 수 있도록 보조
id_key = "doc_id"
# 상위 문서와 하위 문서를 연결할 키값으로 doc_id 사용
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)

#문서 id로 고유한 값을 지정하기 위해 uuid 라이브러리 호출
import uuid
doc_ids = [str(uuid.uuid4()) for _ in docs]

In [18]:
# 하위 청크로 쪼개기 위한 child_text_splitter 지정
child_text_splitter = RecursiveCharacterTextSplitter(chunk_size=400)
# 상위 청크들을 순회하며 하위 청크로 분할한 후 상위 청크 id 상속
sub_docs = []
for i, doc in enumerate(docs):
    _id = doc_ids[i]
    _sub_docs = child_text_splitter.split_documents([doc])
    for _doc in _sub_docs:
        _doc.metadata[id_key] = _id
    sub_docs.extend(_sub_docs)
#vectorstore에 하위 청크 추가
retriever.vectorstore.add_documents(sub_docs)
#docstore에 상위청크 저장할 때, doc_ids 지정
retriever.docstore.mset(list(zip(doc_ids, docs)))

In [19]:
# Vectorstore alone retrieves the small chunks
print("[하위 청크] \n")
print(retriever.vectorstore.similarity_search("국민의 권리")[0].page_content)
print("-"*50)
print("[상위 청크] \n")
print(retriever.invoke("국민의 권리")[0].page_content)

[하위 청크] 

③공공필요에 의한 재산권의 수용ㆍ사용 또는 제한 및 그에 대한 보상은 법률로써 하되, 정당한 보상을 지급하여야
한다.
 
제24조 모든 국민은 법률이 정하는 바에 의하여 선거권을 가진다.
 
제25조 모든 국민은 법률이 정하는 바에 의하여 공무담임권을 가진다.
 
제26조 ①모든 국민은 법률이 정하는 바에 의하여 국가기관에 문서로 청원할 권리를 가진다.
②국가는 청원에 대하여 심사할 의무를 진다.
 
제27조 ①모든 국민은 헌법과 법률이 정한 법관에 의하여 법률에 의한 재판을 받을 권리를 가진다.
②군인 또는 군무원이 아닌 국민은 대한민국의 영역 안에서는 중대한 군사상 기밀ㆍ초병ㆍ초소ㆍ유독음식물공급
ㆍ포로ㆍ군용물에 관한 죄중 법률이 정한 경우와 비상계엄이 선포된 경우를 제외하고는 군사법원의 재판을 받지
--------------------------------------------------
[상위 청크] 

법제처                                                            3                                                       국가법령정보센터
대한민국헌법
제21조 ①모든 국민은 언론ㆍ출판의 자유와 집회ㆍ결사의 자유를 가진다.
②언론ㆍ출판에 대한 허가나 검열과 집회ㆍ결사에 대한 허가는 인정되지 아니한다.
③통신ㆍ방송의 시설기준과 신문의 기능을 보장하기 위하여 필요한 사항은 법률로 정한다.
④언론ㆍ출판은 타인의 명예나 권리 또는 공중도덕이나 사회윤리를 침해하여서는 아니된다. 언론ㆍ출판이 타인의
명예나 권리를 침해한 때에는 피해자는 이에 대한 피해의 배상을 청구할 수 있다.
 
제22조 ①모든 국민은 학문과 예술의 자유를 가진다.
②저작자ㆍ발명가ㆍ과학기술자와 예술가의 권리는 법률로써 보호한다.
 
제23조 ①모든 국민의 재산권은 보장된다. 그 내용과 한계는 법률로 정한다.
②재산권의 행사는 공공복리에 적합하도록 하여야 한다.
③공공필요에 의한 재산권의 수용

### **컨텍스트 재정렬, Long-Context Reorder**

**[Long-Context Reorder 없이 유사 문서 출력]**

In [20]:
#Chroma dimension 관련 에러 발생 시 실행
# Chroma().delete_collection()

In [21]:
from langchain.chains import LLMChain, StuffDocumentsChain
from langchain_chroma import Chroma
from langchain.document_transformers import (
    LongContextReorder,
)
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI

Chroma().delete_collection()

# 한글 임베딩 모델 선언
model_name = "jhgan/ko-sbert-nli"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
embedding = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

texts = [
    "바스켓볼은 훌륭한 스포츠입니다.",
    "플라이 미 투 더 문은 제가 가장 좋아하는 노래 중 하나입니다.",
    "셀틱스는 제가 가장 좋아하는 팀입니다.",
    "이것은 보스턴 셀틱스에 관한 문서입니다."
    "저는 단순히 영화 보러 가는 것을 좋아합니다",
    "보스턴 셀틱스가 20점차로 이겼어요",
    "이것은 그냥 임의의 텍스트입니다.",
    "엘든 링은 지난 15 년 동안 최고의 게임 중 하나입니다.",
    "L. 코넷은 최고의 셀틱스 선수 중 한 명입니다.",
    "래리 버드는 상징적인 NBA 선수였습니다.",
]
# Chroma Retriever 선언(10개의 유사 문서 출력)
retriever = Chroma.from_texts(texts, embedding=embedding).as_retriever(
    search_kwargs={"k": 10}
)
query = "셀틱에 대해 설명해줘"

# 유사도 기준으로 검색 결과 출력
docs = retriever.invoke(query)
docs

[Document(id='f29118fa-1241-49b0-bdc6-ae49922a6f58', metadata={}, page_content='L. 코넷은 최고의 셀틱스 선수 중 한 명입니다.'),
 Document(id='18ddfc03-9bb2-458a-ad7a-e572af1f83b0', metadata={}, page_content='셀틱스는 제가 가장 좋아하는 팀입니다.'),
 Document(id='f5ba8274-b60d-4ee4-8925-bf6779384d6a', metadata={}, page_content='이것은 그냥 임의의 텍스트입니다.'),
 Document(id='64b24f09-ca80-479a-a831-f2ce8fdf3dcb', metadata={}, page_content='이것은 보스턴 셀틱스에 관한 문서입니다.저는 단순히 영화 보러 가는 것을 좋아합니다'),
 Document(id='c8d75e41-59df-414a-a5a0-9ab4e6f65c66', metadata={}, page_content='바스켓볼은 훌륭한 스포츠입니다.'),
 Document(id='6cc3c024-b466-443e-9c8a-ca0dbd60f93f', metadata={}, page_content='보스턴 셀틱스가 20점차로 이겼어요'),
 Document(id='2e15249e-fa97-48d4-93ae-78669d137bb5', metadata={}, page_content='플라이 미 투 더 문은 제가 가장 좋아하는 노래 중 하나입니다.'),
 Document(id='a24e47ea-6c44-4026-b46a-2a3e832a9d96', metadata={}, page_content='엘든 링은 지난 15 년 동안 최고의 게임 중 하나입니다.'),
 Document(id='b3a2d9aa-6f88-4c63-b10b-189cf7f7052d', metadata={}, page_content='래리 버드는 상징적인 NBA 선수였습니다.')]

**[Long-Context Reorder 활용하여 유사 문서 출력]**

In [22]:
#LongContextReorder 선언
reordering = LongContextReorder()
#검색된 유사문서 중 관련도가 높은 문서를 맨앞과 맨뒤에 재정배치
reordered_docs = reordering.transform_documents(docs)
reordered_docs

[Document(id='f29118fa-1241-49b0-bdc6-ae49922a6f58', metadata={}, page_content='L. 코넷은 최고의 셀틱스 선수 중 한 명입니다.'),
 Document(id='f5ba8274-b60d-4ee4-8925-bf6779384d6a', metadata={}, page_content='이것은 그냥 임의의 텍스트입니다.'),
 Document(id='c8d75e41-59df-414a-a5a0-9ab4e6f65c66', metadata={}, page_content='바스켓볼은 훌륭한 스포츠입니다.'),
 Document(id='2e15249e-fa97-48d4-93ae-78669d137bb5', metadata={}, page_content='플라이 미 투 더 문은 제가 가장 좋아하는 노래 중 하나입니다.'),
 Document(id='b3a2d9aa-6f88-4c63-b10b-189cf7f7052d', metadata={}, page_content='래리 버드는 상징적인 NBA 선수였습니다.'),
 Document(id='a24e47ea-6c44-4026-b46a-2a3e832a9d96', metadata={}, page_content='엘든 링은 지난 15 년 동안 최고의 게임 중 하나입니다.'),
 Document(id='6cc3c024-b466-443e-9c8a-ca0dbd60f93f', metadata={}, page_content='보스턴 셀틱스가 20점차로 이겼어요'),
 Document(id='64b24f09-ca80-479a-a831-f2ce8fdf3dcb', metadata={}, page_content='이것은 보스턴 셀틱스에 관한 문서입니다.저는 단순히 영화 보러 가는 것을 좋아합니다'),
 Document(id='18ddfc03-9bb2-458a-ad7a-e572af1f83b0', metadata={}, page_content='셀틱스는 제가 가장 좋아하는 팀입니다.')]